In [5]:
!pip install keras
!pip install tensorflow

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\usrDataScience\\anaconda3\\Lib\\site-packages\\~umpy\\core\\_multiarray_tests.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for jax: filename=jax-0.4.10-py3-none-any.whl size=1480617 sha256=5b4d1069df69980981053b6ce90240c9e9d5bc7a200825ed43b834a226fb8155
  Stored in directory: c:\users\xdashduck\appdata\local\pip\cache\wheels\e5\6c\70\7c6be85fa56f05480fe043bdf0d4f6ec316b122be21e098066
Successfully built jax
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5


# Определение болезней растений с помощью Сиамской модели НС

Цель сиамской сети в том, чтобы понять являются ли два растения похожими или не похожими между собой.

Для обучения сиамской сети нужны данные в виде "пар - метки". Из пары изображений мы передаем одно изображение в сеть А, а другое в сеть Б. Роль этих двух сетей заключается только в извлечении векторов признаков входных изображений. 

Что использовалось?
Два слоя свертки с функцией активации relu для извлечения признаков, которые в последствии мы передаем в виде вектора признаков в общую функцию, измеряющая сходство. Сходство измеряется с помощью евклидово расстояния. Как итог обучение происходит в виде передачи пар изображений и определение семантического сходства.

**Что такое Сиамская нейронная сеть?**

Сначала мы обучаем изображение последовательности сверточных слоев, объединяя слои и полностью связанные слои, в итоге получаем вектор признаков f (x1).
Затем мы обучаем другое изображение в той же последовательности, чтобы получить другой вектор признаков f(x2). Теперь мы вычисляем d, которое будет расстоянием между каждой из точек вектора признаков f (x1) и вектором признаков f(x2).
Если d мало, мы можем сказать, что оба изображения одинаковы, в противном случае, если d велико, все наоборот.

![Fig 1: A Siamese Neural Network for Image Recognition](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.mdpi.com%2Fsymmetry%2Fsymmetry-10-00385%2Farticle_deploy%2Fhtml%2Fimages%2Fsymmetry-10-00385-g001.png&f=1&nofb=1&ipt=68843a7f00c3a2fa7df9b969299f05123168301ad4b071f637acddef99e9aa58&ipo=images)


**Распознавание изображений с одного кадра**

В чем основа всего? В том, что обучение будет происходить с помощью одного кадра. Т.к. существующим алгоритмам и моделям нужно много данных, хочется чтобы при малом наборе была возможность достоверно определять прогноз. 

Другими словами глубокие сверточные нейронные сети стали современными методами для задач классификации изображений. Одним из самых больших ограничений является то, что они требуют большого количества размеченных данных. Во многих приложениях сбор такого большого количества данных иногда невозможен. One Shot Learning призван решить эту проблему.

На чем основывались :
* [Сиамская сеть для распознования лиц](https://github.com/sudharsan13296/Hands-On-Meta-Learning-With-Python/blob/master/02.%20Face%20and%20Audio%20Recognition%20using%20Siamese%20Networks/2.4%20Face%20Recognition%20Using%20Siamese%20Network.ipynb)
* [О том, почему алгоритм одного кадра это актуально](https://towardsdatascience.com/one-shot-learning-with-siamese-networks-using-keras-17f34e75bb3d)
* [Сиамские нейронные сети для OSL](https://www.katnoria.com/siamese-one-shot/)
* [Сиамские сети и OSL для распознавания выдуманных языков](https://sorenbouma.github.io/blog/oneshot/)

### Импорт

In [6]:
import re
import numpy as np
from PIL import Image

from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras import optimizers

import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 

from keras import callbacks
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
import os
from keras.models import Model,load_model
import json
from keras.models import model_from_json, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
selected_image_size = 224
resize = True
total_sample_size = 10000 # 5k-50k

channel = 1
size = 2

folder_count = 38
image_count = 20 #0-50

if resize == True:
    batch_size=256
else:
    batch_size=64

path =  os.path.join('../input/plantvillage/plantvillage_resize_224/PlantVillage_resize_224/')
print(path)

Теперь мы определяем функцию для считывания нашего входного изображения. Функция read_image принимает входные данные в виде изображения и возвращает массив numpy.

feat_vecs_a и feat_vecs_b являются векторами признаков нашей пары изображений. Затем мы передаем эти векторы признаков для вычисления евклидового расстояния. Далее мы определяем нашу функцию потерь как функцию contrastive_loss и компилируем модель.

In [2]:
def read_image(filename, byteorder='>'):
    
    #сначала мы считываем изображение в буфер
    with open(filename, 'rb') as f:
        buffer = f.read()
    
    #используя regex, мы извлекаем заголовок, ширину, высоту и максимальное значение изображения
    header, width, height, maxval = re.search(
        b"(^P5\s(?:\s*#.*[\r\n])*"
        b"(\d+)\s(?:\s*#.*[\r\n])*"
        b"(\d+)\s(?:\s*#.*[\r\n])*"
        b"(\d+)\s(?:\s*#.*[\r\n]\s)*)", buffer).groups()
    
    #затем мы преобразуем изображение в массив numpy, используя np.frombuffer, 
    # который интерпретирует буфер как одномерный массив
    return np.frombuffer(buffer,
                            dtype='u1' if int(maxval) < 256 else byteorder+'u2',
                            count=int(width)*int(height),
                            offset=len(header)
                            ).reshape((int(height), int(width)))


def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

def compute_accuracy(predictions, labels):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return labels[predictions.ravel() < 0.5].mean()

def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

#### Для примера фото:

In [ ]:
Image.open(path+'s1/1.jpg')

In [ ]:
image = mpimg.imread(path+'s1/1.jpg')
dim1 = image.shape[0]
print('dim1',dim1)
dim2 = image.shape[1]
print('dim2',dim2)

In [ ]:
image.shape

После мы определяем другую функцию get_data для генерации наших данных. Как мы знаем, для сиамской сети данные должны быть в виде пар (подлинных и самозваных) с двоичной меткой.

Сначала мы считываем изображения (img1, img2) из того же каталога и сохраняем их в массиве x_genuine_pair и присваиваем y_genuine значение 1. Далее мы считываем изображения (img1, img2) из другого каталога и сохраняем их в паре x_imposite и присваиваем y_imposite значение 0.

Наконец, мы объединяем обе пары x_genuine_pair, x_imposite к X и y_genuine, y_imposite к Y:

# Для подготовки данных

In [ ]:
def get_data(size, total_sample_size):
    
    image = mpimg.imread(path+'s' + str(1) + '/' + str(1) + '.jpg', 'rw+')

    if resize == True:
        image = image[::size, ::size]
        
    dim1 = image.shape[0]
    dim2 = image.shape[1]

    count = 0

    #инициализация массива numpy в форме [total_sample, no_of_pairs, dim1, dim2]
    x_geuine_pair = np.zeros([total_sample_size, 2, 1, dim1, dim2])

    y_genuine = np.zeros([total_sample_size,1])

    for i in range(folder_count):
        for j in range(int(total_sample_size/folder_count)):
            ind1 = 0
            ind2 = 0

            #считывание изображения из такой же директории (подлинная пара)
            while ind1 == ind2:
                ind1 = np.random.randint(image_count)
                ind2 = np.random.randint(image_count)

            img1 = mpimg.imread(path+'s' + str(i+1) + '/' + str(ind1 + 1) + '.jpg', 'rw+')
            img2 = mpimg.imread(path+'s' + str(i+1) + '/' + str(ind2 + 1) + '.jpg', 'rw+')

            if resize == True:
                img1 = img1[::size, ::size]
                img2 = img2[::size, ::size]

            #сохраняем изображения в инициализированном массиве numpy
            print
            x_geuine_pair[count, 0, 0, :, :] = img1
            x_geuine_pair[count, 1, 0, :, :] = img2

            #поскольку мы рисуем изображения из того же каталога, мы присваиваем метке значение 1. (подлинная пара)
            y_genuine[count] = 1
            count += 1

    count = 0
    x_imposite_pair = np.zeros([total_sample_size, 2, 1, dim1, dim2])
    y_imposite = np.zeros([total_sample_size, 1])

    for i in range(int(total_sample_size/image_count)):
        for j in range(image_count):

            #чтение изображений из другого каталога (поддельные пары)
            while True:
                ind1 = np.random.randint(folder_count)
                ind2 = np.random.randint(folder_count)
                if ind1 != ind2:
                    break

            img1 = mpimg.imread(path+'s' + str(ind1+1) + '/' + str(j + 1) + '.jpg', 'rw+')
            img2 = mpimg.imread(path+'s' + str(ind2+1) + '/' + str(j + 1) + '.jpg', 'rw+')

            if resize == True:
                img1 = img1[::size, ::size]
                img2 = img2[::size, ::size]

            x_imposite_pair[count, 0, 0, :, :] = img1
            x_imposite_pair[count, 1, 0, :, :] = img2
            #поскольку мы рисуем изображения из другого каталога, мы присваиваем метке значение 0. (поддельная пара)
            y_imposite[count] = 0
            count += 1

    #теперь объединим подлинные и поддельные пары, чтобы получить все данные целиком.
    #print(x_geuine_pair.shape)
    #print(x_imposite_pair.shape)
    X = np.concatenate([x_geuine_pair, x_imposite_pair], axis=0)/255
    Y = np.concatenate([y_genuine, y_imposite], axis=0)

    return X, Y
X, Y = get_data(size, total_sample_size)

Теперь мы генерируем наши данные и проверяем их размер. Как вы можете видеть, у нас есть 20 000 точек данных, из которых 10 000 являются подлинными парами, а 10 000 - поддельными парами.

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
sns.countplot(Y[:,0])
plt.show()

Затем мы разделяем наши данные для обучения и тестирования в пропорции 85% обучения и 15% тестирования:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.15)

In [ ]:
print('x_train',x_train.shape)
print('x_test',x_test.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)

Теперь, когда мы успешно сгенерировали наши данные, мы создаем нашу сиамскую сеть. Сначала мы определяем базовую сеть, которая в основном представляет собой сверточную сеть, используемую для извлечения признаков. Мы создаем два сверточных слоя с активациями ReLU и максимальным объединением, за которым следует плоский слой.

# Подготовка Сиамской сети

In [ ]:
def build_base_network(input_shape):
    
    seq = Sequential()
    
    nb_filter = [16, 32, 16]
    kernel_size = 3
    
    
    #сверточный 1
    seq.add(Convolution2D(nb_filter[0], kernel_size, kernel_size, input_shape=input_shape,border_mode='valid', dim_ordering='th'))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))  
    seq.add(Dropout(.25))
    
    #сверточный 2
    seq.add(Convolution2D(nb_filter[1], kernel_size, kernel_size, border_mode='valid', dim_ordering='th'))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='th')) 
    seq.add(Dropout(.25))
    
    #сверточный 2
    seq.add(Convolution2D(nb_filter[2], kernel_size, kernel_size, border_mode='valid', dim_ordering='th'))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='th')) 
    seq.add(Dropout(.25))

    #плоский
    seq.add(Flatten())
    seq.add(Dense(128, activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(50, activation='relu'))
    return seq

Далее мы передаем пару изображений в базовую сеть, которая вернет вложения, то есть векторы объектов:

In [ ]:
input_dim = x_train.shape[2:]
img_a = Input(shape=input_dim)
img_b = Input(shape=input_dim)
print('input_dim',input_dim)

In [ ]:
base_network = build_base_network(input_dim)
feat_vecs_a = base_network(img_a)
feat_vecs_b = base_network(img_b)

In [ ]:
distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([feat_vecs_a, feat_vecs_b])

Теперь мы устанавливаем длину эпохи равной 20 и используем [RMSProp](https://keras.io/api/optimizers/rmsprop/) для оптимизации и определения нашей модели.

In [ ]:
epochs = 20
rms = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)#RMSprop()
rms = RMSprop()

earlyStopping = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=3,
                              verbose=1,
                              restore_best_weights=True)
callback_early_stop_reduceLROnPlateau=[earlyStopping]

In [ ]:
model = Model(input=[img_a, img_b], output=distance)
model.compile(loss=contrastive_loss, optimizer=rms,metrics=[accuracy])
model.summary()

In [ ]:
img_1 = x_train[:, 0]
img2 = x_train[:, 1]
img_1.shape
history = model.fit([img_1, img2], y_train, validation_split=.20,
      batch_size= batch_size, verbose=1, nb_epoch=epochs, callbacks=callback_early_stop_reduceLROnPlateau)

# Option 1: Save Weights + Architecture
model.save_weights('model_weights.h5')
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())
print('saved')

Теперь мы делаем прогнозы на основе тестовых данных. Наконец, мы проверяем точность нашей модели.

# Прогноз

In [ ]:
pred = model.predict([x_test[:, 0], x_test[:, 1]])

print('Accuracy on test set: %0.2f%%' % (100 * compute_accuracy(pred, y_test)))

In [ ]:
pred = model.predict([x_train[:, 0], x_train[:, 1]])

print('* Accuracy on training set: %0.2f%%' % (100  * compute_accuracy(pred, y_train)))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

#точность
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#потери
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

# Тест

In [ ]:
if resize==True:
    selected_image_size = int(selected_image_size/2)
    print('selected_image_size',selected_image_size)

In [ ]:
target_label = 1
values = np.array(y_test[:,0])

target_index = values.tolist().index(target_label)
print(target_index)
print('target_index value : ',y_test[target_index])

In [ ]:
img1 = (x_test[target_index, 0] * 255).astype(np.uint8)
img1 = img1.reshape(selected_image_size,selected_image_size)
print(img1.shape)
img1
plt.imshow(img1)
plt.show()

In [ ]:
img2 = (x_test[target_index, 1] * 255).astype(np.uint8)
img2 = img2.reshape(selected_image_size,selected_image_size)
print(img2.shape)
img2
plt.imshow(img2)
plt.show()

In [ ]:
x_test[target_index:target_index+1, 0].shape

In [ ]:
pred = model.predict([x_test[target_index:target_index+1, 0], x_test[target_index:target_index+1, 1]])
pred = pred < 0.5
print('y_test[target_index]:',y_test[target_index,0]==True,' pred :',pred)